In [17]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

SCROLL_PAUSE_TIME = 1.0

def get_case_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # You'll need to adjust this selector based on where the case text actually is
    case_text = soup.select_one('div.case-details').get_text(strip=True)
    return case_text

def scrape_echr_cases():
    url = "http://hudoc.echr.coe.int/eng#{%22documentcollectionid2%22:[%22COMMUNICATEDCASES%22]}"
    
    driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
    print('driver up')
    driver.get(url)
    
    # Wait for the page to load
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "result-item")))
    
    cases = []
    last_height = driver.execute_script("return document.body.scrollHeight")
    print(f'current height {last_height}')
    page_idx = 1
    
    while True:
        print(f'page {page_idx}')
        page_idx+=1
        
        # Parse the current page
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Find all case elements
        case_elements = soup.find_all('div', class_='result-item')
        
        for case in case_elements:
            case_title = case.find('a', class_='document-link headline').text
            case_url = "http://hudoc.echr.coe.int" + case.find('a', class_='document-link headline')['href']
            case_number = case.find('span', class_='column summaryblock').text.strip()
            case_date = case.find('span', class_='column dateColumn').text.strip()
            
            cases.append({
                'title': case_title,
                'url': case_url,
                'number': case_number,
                'date': case_date,
                # 'text': get_case_data(case_url)  # Uncomment this if you want to fetch full text
            })
        
        # scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Calculate new scroll heig
        new_height = driver.execute_script("return document.body.scrollHeight")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        print(f'new height {new_height}')
        # If heights are the same, it means we've reached the end of the page
        if new_height == last_height:
            break
        last_height = new_height
        
        print(f'collected meta data of {len(case_elements)} cases')
    
    driver.quit()
    return cases

In [18]:
# Run the scraper
all_cases = scrape_echr_cases()

driver up
current height 2013
page 1
new height 3726
collected meta data of 20 cases
page 2
new height 5439
collected meta data of 40 cases
page 3
new height 7152
collected meta data of 60 cases
page 4
new height 8865
collected meta data of 80 cases
page 5
new height 10578
collected meta data of 100 cases
page 6
new height 10578


In [7]:
len(all_cases)

20

In [10]:
url = "http://hudoc.echr.coe.int/eng#{%22documentcollectionid2%22:[%22COMMUNICATEDCASES%22]}"

driver = webdriver.Chrome()  # Make sure you have chromedriver installed and in PATH
print('driver up')
driver.get(url)

# Wait for the page to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "result-item")))

cases = []
last_height = driver.execute_script("return document.body.scrollHeight")
print(f'current height {last_height}')
page_idx = 1

driver up
current height 2013


In [12]:
# Calculate new scroll height and compare with last scroll height
new_height = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
print(f'new height {new_height}')

new height None


In [19]:
SCROLL_PAUSE_TIME = 0.5
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)
    
    if new_height == last_height:
        break

MaxRetryError: HTTPConnectionPool(host='localhost', port=55266): Max retries exceeded with url: /session/40d748b9da0674367a7bec6c48253d6c/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x107d0f750>: Failed to establish a new connection: [Errno 61] Connection refused'))